In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from xml.dom import minidom

In [ ]:
#Show the images
def img_show(img, cfg, weights):
    net = cv.dnn_DetectionModel(cfg, weights)
    net.setInputSize(608, 608)
    net.setInputScale(1.0 / 255)
    net.setInputSwapRB(True)
    with open('./model/obj.names', 'rt') as f:
        names = f.read().rstrip('\n').split('\n')
    
    img = cv.imread('./test/'+ img + '.jpg') 
    classes, confidences, boxes = net.detect(img, confThreshold=0.1, nmsThreshold=0.45)

    for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
        label = '%.2f' % confidence
        label = '%s: %s' % (names[classId], label)
        labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        left, top, width, height = box
        top = max(top, labelSize[1])
        cv.rectangle(img, box, color=(0, 255, 0), thickness=3)
        cv.rectangle(img, (left, top - labelSize[1]), (left + labelSize[0], top + baseLine), (0, 255, 0), cv.FILLED)
        cv.putText(img, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
        h,w = img.shape[:2]
        cv.line(img, (w//4*1, 0), (w//4*1, h), (1,0,0), 2)
        cv.line(img, (w//4*2, 0), (w//4*2, h), (1,0,0), 2)
        cv.line(img, (w//4*3, 0), (w//4*3, h), (1,0,0), 2)
        cv.line(img, (0, h//2), (w, h//2), (1,0,0), 2)
    
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

In [ ]:
### Return：the grid where the four vertices of each boX fall
## Set a threshold and do not return grids whose proportion is too small.
def find_grid_mod(img, cfg, weights ):
    net = cv.dnn_DetectionModel(cfg, weights)
    net.setInputSize(608, 608)
    net.setInputScale(1.0 / 255)
    net.setInputSwapRB(True)
    with open('./model/obj.names', 'rt') as f:
        names = f.read().rstrip('\n').split('\n')
    
    img = cv.imread('./test/'+ img + '.jpg') 
    classes, confidences, boxes = net.detect(img, confThreshold=0.1, nmsThreshold=0.45)
    
    ###find all vertices
    h,w = img.shape[:2]
    alist=[]
    for n in boxes:
        points=[]
        points.append([n[0],n[1]])
        points.append([n[0]+n[2],n[1]])
        points.append([n[0],n[1]+n[3]])
        points.append([n[0]+n[2],n[1]+n[3]])
        alist.append(points)
    
    ### all_grid: each box is equipped with a top grid
    all_grid = []
    for points in alist:
        grid = set()
        for point in points:
            x = point[0] / w
            y = point[1] / h
            for n in range(4):
                if x >= (0.25*n) and x <= (0.25*(n+1)):
                    if y <= 0.5:
                        grid.add(n+1)
                    else:
                        grid.add(n+5)
        all_grid.append(grid)
    
    ### Determine the proportion of the distance from the vertex to the grid border 
    ### to the overall box size (if it is less than 0.15, exclude the grid)
    answer = set()
    for points , grid in zip(alist,all_grid):
        if len(grid) == 1 :
            answer = answer | grid
        elif len(grid) == 2:
            l_grid = list(grid)
            l_grid.sort()
            if l_grid[1] == l_grid[0] + 1:
                bound = (l_grid[0] % 4 ) * (w/4)
                if (abs(bound - points[0][0]))/(points[1][0] -points[0][0]) < 0.2:
                    grid.remove(l_grid[0])
                if (abs(bound - points[1][0]))/(points[1][0] -points[0][0]) < 0.2:
                    grid.remove(l_grid[1])
            elif l_grid[1] == l_grid[0] + 4:
                bound = h / 2
                if abs(bound - points[0][1]) /(points[2][1] -points[0][1]) < 0.2:
                    grid.remove(l_grid[0])
                if abs(bound - points[2][1]) /(points[2][1] -points[0][1]) < 0.2:
                    grid.remove(l_grid[1])
            answer = answer | grid
        elif len(grid) == 4:
            l_grid = list(grid)
            l_grid.sort()
            a = []
            bound_x = (l_grid[0] % 4) * (w/4)
            bound_y = h / 2
            for n,point in enumerate(points):
                if (abs(bound_x - point[0])) /(points[1][0] -points[0][0]) < 0.2 or\
                    (abs(bound_y - point[1])) /(points[2][1] -points[0][1]) < 0.2:
                    a.append(l_grid[n])
            for _ in a:
                grid.remove(_)
            answer = answer | grid
    return answer

In [ ]:
### color for boxes
def draw_grid(img, cfg, weights ):
    net = cv.dnn_DetectionModel(cfg, weights)
    net.setInputSize(608, 608)
    net.setInputScale(1.0 / 255)
    net.setInputSwapRB(True)
    with open('./model/obj.names', 'rt') as f:
        names = f.read().rstrip('\n').split('\n')
    
    img = cv.imread('./test/'+ img + '.jpg') 
    classes, confidences, boxes = net.detect(img, confThreshold=0.1, nmsThreshold=0.45)
    
    ###find all vertices
    h,w = img.shape[:2]
    alist=[]
    for n in boxes:
        points=[]
        points.append([n[0],n[1]])
        points.append([n[0]+n[2],n[1]])
        points.append([n[0],n[1]+n[3]])
        points.append([n[0]+n[2],n[1]+n[3]])
        alist.append(points)
    
    ### all_grid: each box is equipped with a top grid
    all_grid = []
    for points in alist:
        grid = set()
        for point in points:
            x = point[0] / w
            y = point[1] / h
            for n in range(4):
                if x >= (0.25*n) and x <= (0.25*(n+1)):
                    if y <= 0.5:
                        grid.add(n+1)
                    else:
                        grid.add(n+5)
        all_grid.append(grid)
    
    ### Determine the proportion of the distance from the vertex to the grid border 
    ### to the overall box size (if it is less than 0.15, exclude the grid)
    answer = set()
    for points , grid in zip(alist,all_grid):
        if len(grid) == 1 :
            answer = answer | grid
        elif len(grid) == 2:
            l_grid = list(grid)
            l_grid.sort()
            if l_grid[1] == l_grid[0] + 1:
                bound = (l_grid[0] % 4 ) * (w/4)
                if (abs(bound - points[0][0]))/(points[1][0] -points[0][0]) < 0.2:
                    grid.remove(l_grid[0])
                if (abs(bound - points[1][0]))/(points[1][0] -points[0][0]) < 0.2:
                    grid.remove(l_grid[1])
            elif l_grid[1] == l_grid[0] + 4:
                bound = h / 2
                if abs(bound - points[0][1]) /(points[2][1] -points[0][1]) < 0.2:
                    grid.remove(l_grid[0])
                if abs(bound - points[2][1]) /(points[2][1] -points[0][1]) < 0.2:
                    grid.remove(l_grid[1])
            answer = answer | grid
        elif len(grid) == 4:
            l_grid = list(grid)
            l_grid.sort()
            a = []
            bound_x = (l_grid[0] % 4) * (w/4)
            bound_y = h / 2
            for n,point in enumerate(points):
                if (abs(bound_x - point[0])) /(points[1][0] -points[0][0]) < 0.2 or\
                    (abs(bound_y - point[1])) /(points[2][1] -points[0][1]) < 0.2:
                    a.append(l_grid[n])
            for _ in a:
                grid.remove(_)
            answer = answer | grid
    answer = list(answer)
    
    
    ###Color part
    for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
        label = '%.2f' % confidence
        label = '%s: %s' % (names[classId], label)
        labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        left, top, width, height = box
        top = max(top, labelSize[1])
        cv.rectangle(img, box, color=(0, 255, 0), thickness=3)
        cv.rectangle(img, (left, top - labelSize[1]), (left + labelSize[0], top + baseLine), (0, 255, 0), cv.FILLED)
        cv.putText(img, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
        h,w = img.shape[:2]

    for n in answer:
        x1 = (n - 1) % 4
        y1 = (n - 1) // 4
        x2 = x1 + 1
        y2 = y1 + 1
        leftpoint = (x1 * int(w//4),y1 * int(h//2))
        rightpoint = (x2 * int(w//4),y2 * int(h//2))
        zeros1 = np.zeros((img.shape), dtype=np.uint8)
        zeros_mask = cv.rectangle(zeros1, (leftpoint), (rightpoint),
                    color=(0,0,255), thickness=-1 )
        
        ### Color transparency adjustment beta
        alpha, beta, gamma = 1, 0.3, 0
        img = cv.addWeighted(img, alpha, zeros_mask, beta, gamma)
    
    cv.line(img, (w//4*1, 0), (w//4*1, h), (1,0,0), 2)
    cv.line(img, (w//4*2, 0), (w//4*2, h), (1,0,0), 2)
    cv.line(img, (w//4*3, 0), (w//4*3, h), (1,0,0), 2)
    cv.line(img, (0, h//2), (w, h//2), (1,0,0), 2)
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

In [ ]:
### Classify wear distribution
### Category 1: Normal; Category 2 is inner wear; Category 3 is outer wear; Category 4 is front-end wear; 
### Category 5 is heel wear; Category 6 is excessive wear and tear; Category 7 is insufficient wear and tear and cannot be judged
def classified(a):
    inside = {1,2,3,4}
    outside = {5,6,7,8}
    if len(a)==8 or len(a)==7:
        cla = 6   
        
    elif len(a) ==6:
        if {1,4,5,8}.issubset(a):
            cla = 1
        elif inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        else:
            cla = 6
            
    elif len(a) == 5:
        if inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        elif {1,2,5,6}.issubset(a):
            cla = 5
        else:
            cla = 1
            
    elif len(a) == 4:
        if inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        elif len({3,4,7,8}.intersection(a)) == 4:
            cla = 4
        elif len({1,2,5,6}.intersection(a)) == 4 or len({1,2,5,6}.intersection(a)) == 3:
            cla = 5     
        else:
            cla = 1
            
    elif len(a)==2 or len(a)==3:
        if {4,8}.issubset(a):
            cla = 4
        elif {1,5}.issubset(a):
            cla = 5  
        elif inside.issuperset(a): ### If inside is a superset of a
            cla = 2
        elif outside.issuperset(a):
            cla = 3
        else:
            cla = 1
    
    
    else:
        if a.issubset({1,5}):
            cla = 5
        elif a.issubset({4,9}):
            cla = 4
        else:
            cla = 7
    return cla
        
        


In [ ]:
classified({1,2,5,7})

In [ ]:
img = 'test01'
cfg = './model/yolov4.cfg'
weights = './model/yolov4_final.weights'

In [ ]:
img_show(img, cfg, weights) ### Find where the wear is

In [ ]:
find_grid_mod(img, cfg, weights ) ### Find the grid containing the wear frame (needs to be greater than 0.2)

In [ ]:
draw_grid(img, cfg, weights)     ### Color grid

In [ ]:
classified(find_grid_mod(img, cfg, weights ))   ### Classification according to wear grid

In [ ]:
def find_class(img, cfg, weights):
    net = cv.dnn_DetectionModel(cfg, weights)
    net.setInputSize(608, 608)
    net.setInputScale(1.0 / 255)
    net.setInputSwapRB(True)
    with open('./model/obj.names', 'rt') as f:
        names = f.read().rstrip('\n').split('\n')
    
    img = cv.imread('./test/'+ img + '.jpg') 
    classes, confidences, boxes = net.detect(img, confThreshold=0.1, nmsThreshold=0.45)
    
    ###find all vertices
    h,w = img.shape[:2]
    alist=[]
    for n in boxes:
        points=[]
        points.append([n[0],n[1]])
        points.append([n[0]+n[2],n[1]])
        points.append([n[0],n[1]+n[3]])
        points.append([n[0]+n[2],n[1]+n[3]])
        alist.append(points)
    
    ### all_grid: each box is equipped with a top grid
    all_grid = []
    for points in alist:
        grid = set()
        for point in points:
            x = point[0] / w
            y = point[1] / h
            for n in range(4):
                if x >= (0.25*n) and x <= (0.25*(n+1)):
                    if y <= 0.5:
                        grid.add(n+1)
                    else:
                        grid.add(n+5)
        all_grid.append(grid)
    
    ### Determine the proportion of the distance from the vertex to the grid border 
    ### to the overall box size (if it is less than 0.15, exclude the grid)
    answer = set()
    for points , grid in zip(alist,all_grid):
        if len(grid) == 1 :
            answer = answer | grid
        elif len(grid) == 2:
            l_grid = list(grid)
            l_grid.sort()
            if l_grid[1] == l_grid[0] + 1:
                bound = (l_grid[0] % 4 ) * (w/4)
                if (abs(bound - points[0][0]))/(points[1][0] -points[0][0]) < 0.2:
                    grid.remove(l_grid[0])
                if (abs(bound - points[1][0]))/(points[1][0] -points[0][0]) < 0.2:
                    grid.remove(l_grid[1])
            elif l_grid[1] == l_grid[0] + 4:
                bound = h / 2
                if abs(bound - points[0][1]) /(points[2][1] -points[0][1]) < 0.2:
                    grid.remove(l_grid[0])
                if abs(bound - points[2][1]) /(points[2][1] -points[0][1]) < 0.2:
                    grid.remove(l_grid[1])
            answer = answer | grid
        elif len(grid) == 4:
            l_grid = list(grid)
            l_grid.sort()
            a = []
            bound_x = (l_grid[0] % 4) * (w/4)
            bound_y = h / 2
            for n,point in enumerate(points):
                if (abs(bound_x - point[0])) /(points[1][0] -points[0][0]) < 0.2 or\
                    (abs(bound_y - point[1])) /(points[2][1] -points[0][1]) < 0.2:
                    a.append(l_grid[n])
            for _ in a:
                grid.remove(_)
            answer = answer | grid
    a = answer.copy()
    answer = list(answer)    
    
    ###Color part
    for classId, confidence, box in zip(classes.flatten(), confidences.flatten(), boxes):
        label = '%.2f' % confidence
        label = '%s: %s' % (names[classId], label)
        labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        left, top, width, height = box
        top = max(top, labelSize[1])
        cv.rectangle(img, box, color=(0, 255, 0), thickness=3)
        cv.rectangle(img, (left, top - labelSize[1]), (left + labelSize[0], top + baseLine), (0, 255, 0), cv.FILLED)
        cv.putText(img, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
        h,w = img.shape[:2]

    for n in answer:
        x1 = (n - 1) % 4
        y1 = (n - 1) // 4
        x2 = x1 + 1
        y2 = y1 + 1
        leftpoint = (x1 * int(w//4),y1 * int(h//2))
        rightpoint = (x2 * int(w//4),y2 * int(h//2))
        zeros1 = np.zeros((img.shape), dtype=np.uint8)
        zeros_mask = cv.rectangle(zeros1, (leftpoint), (rightpoint),
                    color=(0,0,255), thickness=-1 )
        
        ### Color transparency adjustment beta
        alpha, beta, gamma = 1, 0.3, 0
        img = cv.addWeighted(img, alpha, zeros_mask, beta, gamma)
    
    cv.line(img, (w//4*1, 0), (w//4*1, h), (1,0,0), 2)
    cv.line(img, (w//4*2, 0), (w//4*2, h), (1,0,0), 2)
    cv.line(img, (w//4*3, 0), (w//4*3, h), (1,0,0), 2)
    cv.line(img, (0, h//2), (w, h//2), (1,0,0), 2)
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    
    ###Classify
    inside = {1,2,3,4}
    outside = {5,6,7,8}
    if len(a)==8 or len(a)==7:
        cla = 6   
        
    elif len(a) ==6:
        if {1,4,5,8}.issubset(a):
            cla = 1
        elif inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        else:
            cla = 6
            
    elif len(a) == 5:
        if inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        elif {1,2,5,6}.issubset(a):
            cla = 5
        else:
            cla = 1
            
    elif len(a) == 4:
        if inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        elif {1,2,5,6}.issubset(a):
            cla = 5
        else:
            cla = 1
            
    elif len(a)==2 or len(a)==3:
        if {4,8}.issubset(a):
            cla = 4
        elif {1,5}.issubset(a):
            cla = 5  
        elif inside.issuperset(a): ###If inside is a superset of a
            cla = 2
        elif outside.issuperset(a):
            cla = 3
        else:
            cla = 1
    
    
    else:
        if a.issubset({1,5}):
            cla = 5
        elif a.issubset({4,9}):
            cla = 4
        else:
            cla = 7
    if len(a)==8 or len(a)==7:
        cla = 6   
        
    elif len(a) ==6:
        if {1,4,5,8}.issubset(a):
            cla = 1
        elif inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        else:
            cla = 6
            
    elif len(a) == 5:
        if inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        elif {1,2,5,6}.issubset(a):
            cla = 5
        else:
            cla = 1
            
    elif len(a) == 4:
        if inside.issubset(a):
            cla = 2
        elif outside.issubset(a):
            cla = 3
        elif len({3,4,7,8}.intersection(a)) == 4:
            cla = 4
        elif len({1,2,5,6}.intersection(a)) == 4 or len({1,2,5,6}.intersection(a)) == 3:
            cla = 5     
        else:
            cla = 1
            
    elif len(a)==2 or len(a)==3:
        if {4,8}.issubset(a):
            cla = 4
        elif {1,5}.issubset(a):
            cla = 5  
        elif inside.issuperset(a): ###If inside is a superset of a
            cla = 2
        elif outside.issuperset(a):
            cla = 3
        else:
            cla = 1
    
    
    else:
        if a.issubset({1,5}):
            cla = 5
        elif a.issubset({4,9}):
            cla = 4
        else:
            cla = 7
    print("It belongs to category {} ".format(cla))
    print("================================================")
    
        

In [ ]:
find_class(img, cfg, weights)